# 3.Visualization 

In [1]:
import numpy as np
import pandas as pd
import sqlite3
import sys
import sqlalchemy
from ipywidgets import *
import ipywidgets as widgets
from IPython.display import display
from traitlets import link

In [2]:
from bokeh.plotting import figure
from bokeh.io import output_file, show, save, output_notebook
from bokeh.models import Range1d, NumeralTickFormatter, HoverTool, ColumnDataSource
from bokeh.sampledata.iris import flowers
output_notebook()

Loading BokehJS ...

## 3.1 Data Prep 

In [3]:
#Loading database after selection
## Connecting database
db_conn = sqlite3.connect('AVA_FY.db')
db_cur = db_conn.cursor()
#load all tables
db_cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
metric_tables = db_cur.fetchall()
metric_names = [i[0] for i in metric_tables]
var_names = [name.replace(" ", "_") for name in metric_names]
df_ava = {}
for i in range(len(var_names)):
    sql_query = 'SELECT * FROM "{table_name}"'.format(table_name=metric_names[i])
    key = metric_names[i]
    value = pd.read_sql(sql_query, db_conn)
    df_ava[key] = value

In [4]:
## Create Selector
sector_list=list(set(df_ava["Total Revenue"]["AV Sector"]))
sector_list=list(np.sort(sector_list))
country_list=list(set(df_ava["Total Revenue"]["Country"]))
country_list=list(np.sort(country_list))
column_names = list(df_ava["Total Revenue"].columns)
year_list = column_names[column_names.index("FY1998"):]

filter_idea_data = widgets.Accordion(
    children=[widgets.Dropdown(description="Select Sector",options=sector_list),
              widgets.Dropdown(description="Select Country",options=country_list),
              widgets.SelectMultiple(description="Select Years To Be Included",options=year_list),
             ])
filter_idea_data.set_title(0, 'Select Sector')
filter_idea_data.set_title(1, 'Select Country')
filter_idea_data.set_title(2, 'Select Year')

filter_idea_data

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


In [7]:
## Reload Database again after filtering
print(filter_idea_data.children[0].value)
print(filter_idea_data.children[1].value)
print(filter_idea_data.children[2].value)
# Define info list about selected companies
info_list=["Company Name", "Exchange:Ticker", "Country", "Region", "AV Industry", "AV Sector"]
year_label=list(filter_idea_data.children[2].value)

Auto OEM
United States
('FY2011', 'FY2012', 'FY2013', 'FY2014', 'FY2015')


In [10]:
## Reload Database
new_db_info = {}
new_db_data = {}
new_db={}
for table in list(df_ava.keys()):
    temp_df=df_ava[table][df_ava[table]["AV Sector"]==filter_idea_data.children[0].value]
#     temp_df=temp_df[temp_df["Country"]==filter_idea_data.children[1].value]
    temp_df=temp_df.loc[:,info_list]
    new_db_info[table]=temp_df


for table in list(df_ava.keys()):
    temp_df=df_ava[table][df_ava[table]["AV Sector"]==filter_idea_data.children[0].value]
#     temp_df=temp_df[temp_df["Country"]==filter_idea_data.children[1].value]
    temp_df=temp_df.loc[:,["Company Name"]+year_label]
    new_db_data[table]=temp_df

for table in list(df_ava.keys()):
    temp_df=df_ava[table][df_ava[table]["AV Sector"]==filter_idea_data.children[0].value]
#     temp_df=temp_df[temp_df["Country"]==filter_idea_data.children[1].value]
    temp_df=temp_df.loc[:,info_list+year_label]
    new_db[table]=temp_df

In [11]:
temp_dict={}
for i in list(new_db_data.keys()):
    temp_var = new_db_data.ix[i]
    temp_dict[temp_var[0]] = list(temp_var[1:])

AttributeError: 'dict' object has no attribute 'ix'

In [12]:
gm_cogs = temp_dict["General Motors Company (NYSE:GM)"]
f_cogs = temp_dict["Ford Motor Company (NYSE:F)"]
tesla_cogs = temp_dict["Tesla Motors, Inc. (NasdaqGS:TSLA)"]

KeyError: 'General Motors Company (NYSE:GM)'

In [208]:
f.square(x, y, size=10, color="olive", alpha=0.5)
show(f)

In [209]:
#style the tools
TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select"
f1 = figure(x_range=year_label)
f1.circle(x=year_label, y=gm_cogs, color="olive", alpha=0.5)
f1.line(x=year_label, y=gm_cogs, color="black", alpha=0.5)
f1.circle(x=year_label, y=f_cogs, color="olive", alpha=0.5)
f1.line(x=year_label, y=f_cogs, color="black", alpha=0.5)
f1.circle(x=year_label, y=tesla_cogs, color="olive", alpha=0.5)
f1.line(x=year_label, y=tesla_cogs, color="black", alpha=0.5)
show(f1)
f1.toolbar.logo=None

In [193]:
y

['104372', '113345', '112578', '125234', '123516', '124041']